In [1]:
import sys, subprocess, importlib
try:
    import Cython
    print("Versão do Cython:", Cython.__version__)
except ImportError:
    print("Cython não está instalado neste kernel:", sys.executable)


Versão do Cython: 3.0.12


In [2]:
from Cython.Compiler import Options

Options.language_level = 3 # Python 3
Options.boundscheck = False # Desliga a checagem de limites em acessos a arrays (loops de acesso a vetor ficam bem mais rápidos)
Options.wraparound = False # Desliga o suporte a índices negativos a[-1] (aumenta desempenho)
Options.cdivision = True # Usa a divisão inteira de C para tipos inteiros (divisão de inteiros fica mais rápida, pois evita verificações de exceção)

%load_ext Cython

In [3]:
def soma_py(n: int) -> int:
    acc = 0
    for i in range(n):
        acc += i
        
    return acc

In [4]:
%%cython
# cython: language_level=3, boundscheck=False, wraparound=False, cdivision=True

def soma_cy(int n):
    cdef long acc = 0
    cdef int i
    for i in range(n):
        acc += i
    return acc


In [5]:
import timeit, statistics

N = 10_000_000        # tamanho do problema
R = 5                 # quantas repetições queremos medir

# helper genérico para qualquer função  -----------------------------
def bench(fn, *args, repeat=R, number=1):
    timings = timeit.repeat(
        lambda: fn(*args),
        repeat=repeat,
        number=number,
    )
    
    avg = statistics.mean(timings)
    print(f"{fn.__name__:8s}: média {avg:.6f} s  "f"(±{statistics.stdev(timings):.6f})")
    return avg

# execução -----------------------------------------------------------
t_py = bench(soma_py, N)
t_cy = bench(soma_cy, N)

print(f"\nSpeed-up ≈ {t_py / t_cy:.1f}×")


soma_py : média 0.488434 s  (±0.076273)
soma_cy : média 0.004472 s  (±0.000569)

Speed-up ≈ 109.2×
